In [1]:
from __future__ import print_function, division
import torch
import torch
import torch.nn as nn
import torch.optim as optim
import data as dt
import model_4_o as md
import copy
from pytorch_ssim import ssim
from torch.utils.data import DataLoader
# e/d + i
# best mode so far

In [ ]:
# print gpu
torch.cuda.set_device(1)
currentDevice = torch.cuda.current_device()
print("Current GPU: " + str(currentDevice))
print(str(torch.cuda.device_count()))
print(str(torch.cuda.get_device_capability(currentDevice)))
print(torch.__version__)

In [2]:
USE_GPU = 1
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda:1')
else:
    device = "cpu"
print(device)

cuda:1


In [ ]:
# generate csv file, run only for the first time
# dt.generate_csv()

In [3]:
csvFilePath = dt.get_csv_path()
transformed_dataset = dt.HE_SHG_Dataset(csv_file=csvFilePath,
                                               transform=dt.Compose([                                              
                                               dt.Rescale(96),                                     
                                               dt.Normalize(),
                                               dt.ToTensor()
                                           ]))
# TODO: change the normalization parameters

In [4]:
# batchsize 32->16
dataloader = DataLoader(transformed_dataset, batch_size=1200,
                        shuffle=True, num_workers=0)

In [ ]:
# TODO: insert back mean and variance to plot the image appropriately
dt.show_patch(dataloader) 

In [5]:
import os
cwd = os.getcwd();
path = os.path.join(cwd, 'Saved model', 'encoderresinfo.pth')
model = md.Net()
model.load_state_dict(torch.load(path))
model.eval()


Net(
  (block0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
  )
  (block1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.1)
  )
  (info1): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
  )
  (block2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [6]:
criterionMSE = nn.MSELoss()
criterionMSE = criterionMSE.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
def train(epoch, p, windowsize):
    epoch_loss = 0
    for iteration, batch in enumerate(dataloader):
        input, target = batch['input'].to(device), batch['output'].to(device)

        optimizer.zero_grad()
        output = model(input)
        targetf = target.float()
        targetf = targetf[:, None]
        
        lossMSE = criterionMSE(output, targetf)      
        lossSSIM = 1-ssim(output, targetf, window_size=windowsize)
        
        # 0.75->0.4 after 3 epochs
        loss = p*lossMSE + (1-p)*lossSSIM
        combineLoss = p*lossMSE.item() + (1-p)*lossSSIM.item()
        
        epoch_loss = epoch_loss + combineLoss
        loss.backward()
        optimizer.step()
    
        if iteration%50 == 0:
            print("lossMSE: " + str(lossMSE.item()) +
                  " " + "lossSSIM: " + str(lossSSIM.item()))
            print("===> Epoch[{}]({}/{}): Loss: {:.4f}".format(epoch, iteration, len(dataloader), loss.item()))

    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(dataloader)))


In [ ]:
# def test():
#     avg_psnr = 0
#     with torch.no_grad():
#         for batch in testing_data_loader:
#             input, target = batch[0].to(device), batch[1].to(device)

#             prediction = model(input)
#             mse = criterion(prediction, target)
#             psnr = 10 * log10(1 / mse.item())
#             avg_psnr += psnr
#     print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(testing_data_loader)))


In [ ]:
# def checkpoint(epoch):
#     model_out_path = "model_epoch_{}.pth".format(epoch)
#     torch.save(model, model_out_path)
#     print("Checkpoint saved to {}".format(model_out_path))

In [ ]:
for epoch in range(1, 10 + 1):
    train(epoch)
#     test()
#     checkpoint(epoch)
#     wanna loss < 0.06

In [8]:
import matplotlib.pyplot as plt
from torchvision import transforms, utils
def test():
    avg_psnr = 0
    with torch.no_grad():
        for iteration, batch in enumerate(dataloader):
            input, target = batch['input'].to(device), batch['output'].to(device)
                
            prediction = model(input)

            target = target.float()

            outdataloader = {'input':prediction,'output':target}
            
            print(outdataloader['input'].size(), 
                      outdataloader['output'].size())

            plt.figure()
            input_batch, label_batch = outdataloader['input'], outdataloader['output']
            batch_size = 32
            im_size = input_batch.size(2)
            label_batch=label_batch.reshape([batch_size,1,im_size,im_size])
            print(label_batch.size())
#             for img in input_batch:
#                 for t, m, s in zip(img, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]):
#                     t.mul_(s).add_(m)
                            
#             for img in label_batch:
#                 for t, m, s in zip(img, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]):
#                     t.mul_(s).add_(m)                           
            grid = utils.make_grid(input).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))
            plt.figure()
    
            grid = utils.make_grid(input_batch).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))
            plt.figure()

            grid = utils.make_grid(label_batch).cpu()
            plt.imshow(grid.numpy().transpose((1, 2, 0)))

            plt.axis('off')
            plt.ioff()
            plt.show()
            
            targetf = target[:, None]
            
            lossMSE = criterionMSE(prediction, targetf)      
            lossSSIM = -ssim(prediction, targetf)
        
            p = 0.25
            loss = p*lossMSE + (1-p)*lossSSIM
            combineLoss = p*lossMSE.item() + (1-p)*lossSSIM.item()
#             mse = criterion(prediction, target.float())

            psnr = 10 * torch.log10(1 / loss)
            avg_psnr += psnr
            if iteration == 16:
                break
    print("===> Avg. PSNR: {:.4f} dB".format(avg_psnr / len(dataloader)))

In [ ]:
test()

In [ ]:
l = 0.0001
p = 0.5
windowsize = 3
for epoch in range(1, 18 + 1):
    if epoch%2 == 0:
        windowsize = windowsize+1
        p = p*0.8
        l = l*0.2
        if windowsize == 0:
            windowsze = 1
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

In [ ]:
test()

In [ ]:
import os
cwd = os.getcwd();
path = os.path.join(cwd, 'Saved model', 'encoderresinfo.pth')
torch.save(model.state_dict(), path)

In [ ]:
l = 0.0001
p = 0
windowsize = 4
for epoch in range(1, 20 + 1):
    if epoch%5 == 0:
        windowsize = windowsize+1
        p = p*1
        l = l*0.2
        if windowsize == 0:
            windowsze = 1
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

In [ ]:
l = 0.00001
p = 0
windowsize = 10
for epoch in range(1, 20 + 1):
    if epoch%5 == 0:
        windowsize = windowsize+1
        p = p*1
        l = l*0.2
        if windowsize >= 20:
            windowsze = 20
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

In [ ]:
test()

In [9]:
gpu_ids = [1, 2, 3, 4, 5, 6]
torch.cuda.set_device(gpu_ids[0])
model = torch.nn.DataParallel(model, device_ids=gpu_ids)
model.cuda()

DataParallel(
  (module): Net(
    (block0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
    (block1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.1)
    )
    (info1): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
    )
    (block2): Sequential(
      (0): Co

In [10]:
l = 0.0001
p = 0
windowsize = 10
for epoch in range(1, 20 + 1):
    if epoch%5 == 0:
        windowsize = windowsize+1
        p = p*1
        l = l*1
        if windowsize >= 20:
            windowsze = 20
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

lossMSE: 0.0061826989986002445 lossSSIM: 0.26286572217941284
===> Epoch[1](0/676): Loss: 0.2629
lossMSE: 0.007225800771266222 lossSSIM: 0.26099473237991333
===> Epoch[1](50/676): Loss: 0.2610
lossMSE: 0.006101866252720356 lossSSIM: 0.2660636901855469
===> Epoch[1](100/676): Loss: 0.2661
lossMSE: 0.006052036304026842 lossSSIM: 0.2484944462776184
===> Epoch[1](150/676): Loss: 0.2485
lossMSE: 0.006251385901123285 lossSSIM: 0.2548221945762634
===> Epoch[1](200/676): Loss: 0.2548
lossMSE: 0.007263422477990389 lossSSIM: 0.2613692879676819
===> Epoch[1](250/676): Loss: 0.2614
lossMSE: 0.0062272027134895325 lossSSIM: 0.2564438581466675
===> Epoch[1](300/676): Loss: 0.2564
lossMSE: 0.006103506311774254 lossSSIM: 0.2573243975639343
===> Epoch[1](350/676): Loss: 0.2573
lossMSE: 0.0060762688517570496 lossSSIM: 0.2600712180137634
===> Epoch[1](400/676): Loss: 0.2601
lossMSE: 0.006617461796849966 lossSSIM: 0.2579900026321411
===> Epoch[1](450/676): Loss: 0.2580
lossMSE: 0.00664266524836421 lossSSIM:

===> Epoch 6 Complete: Avg. Loss: 0.2620
lossMSE: 0.0074668582528829575 lossSSIM: 0.2746887803077698
===> Epoch[7](0/676): Loss: 0.2747
lossMSE: 0.006691941060125828 lossSSIM: 0.26588064432144165
===> Epoch[7](50/676): Loss: 0.2659
lossMSE: 0.007539073936641216 lossSSIM: 0.2672088146209717
===> Epoch[7](100/676): Loss: 0.2672
lossMSE: 0.006694955285638571 lossSSIM: 0.2552531957626343
===> Epoch[7](150/676): Loss: 0.2553
lossMSE: 0.007189897820353508 lossSSIM: 0.26018548011779785
===> Epoch[7](200/676): Loss: 0.2602
lossMSE: 0.006568498443812132 lossSSIM: 0.2538940906524658
===> Epoch[7](250/676): Loss: 0.2539
lossMSE: 0.007261935155838728 lossSSIM: 0.26421958208084106
===> Epoch[7](300/676): Loss: 0.2642
lossMSE: 0.006038019433617592 lossSSIM: 0.2586071491241455
===> Epoch[7](350/676): Loss: 0.2586
lossMSE: 0.007561223115772009 lossSSIM: 0.27107447385787964
===> Epoch[7](400/676): Loss: 0.2711
lossMSE: 0.005891743581742048 lossSSIM: 0.2527363896369934
===> Epoch[7](450/676): Loss: 0.25

lossMSE: 0.006793653592467308 lossSSIM: 0.2604805827140808
===> Epoch[12](650/676): Loss: 0.2605
===> Epoch 12 Complete: Avg. Loss: 0.2619
lossMSE: 0.007115494925528765 lossSSIM: 0.2633376121520996
===> Epoch[13](0/676): Loss: 0.2633
lossMSE: 0.006557554937899113 lossSSIM: 0.260015070438385
===> Epoch[13](50/676): Loss: 0.2600
lossMSE: 0.006561868358403444 lossSSIM: 0.25471264123916626
===> Epoch[13](100/676): Loss: 0.2547
lossMSE: 0.007131645921617746 lossSSIM: 0.27166199684143066
===> Epoch[13](150/676): Loss: 0.2717
lossMSE: 0.007268934510648251 lossSSIM: 0.26648807525634766
===> Epoch[13](200/676): Loss: 0.2665
lossMSE: 0.0054704053327441216 lossSSIM: 0.2532948851585388
===> Epoch[13](250/676): Loss: 0.2533
lossMSE: 0.0072773052379488945 lossSSIM: 0.26844966411590576
===> Epoch[13](300/676): Loss: 0.2684
lossMSE: 0.006924827117472887 lossSSIM: 0.27977538108825684
===> Epoch[13](350/676): Loss: 0.2798
lossMSE: 0.006973852403461933 lossSSIM: 0.25744229555130005
===> Epoch[13](400/676

lossMSE: 0.006061266642063856 lossSSIM: 0.2543175220489502
===> Epoch[18](550/676): Loss: 0.2543
lossMSE: 0.006949311122298241 lossSSIM: 0.2588895559310913
===> Epoch[18](600/676): Loss: 0.2589
lossMSE: 0.007043274585157633 lossSSIM: 0.2649775743484497
===> Epoch[18](650/676): Loss: 0.2650
===> Epoch 18 Complete: Avg. Loss: 0.2618
lossMSE: 0.0068953437730669975 lossSSIM: 0.2668514847755432
===> Epoch[19](0/676): Loss: 0.2669
lossMSE: 0.006092885509133339 lossSSIM: 0.25203579664230347
===> Epoch[19](50/676): Loss: 0.2520
lossMSE: 0.005996382795274258 lossSSIM: 0.2530503273010254
===> Epoch[19](100/676): Loss: 0.2531
lossMSE: 0.007369290105998516 lossSSIM: 0.27218031883239746
===> Epoch[19](150/676): Loss: 0.2722
lossMSE: 0.006428185384720564 lossSSIM: 0.258600652217865
===> Epoch[19](200/676): Loss: 0.2586
lossMSE: 0.006828335579484701 lossSSIM: 0.25857383012771606
===> Epoch[19](250/676): Loss: 0.2586
lossMSE: 0.006508093327283859 lossSSIM: 0.2555738091468811
===> Epoch[19](300/676): L

In [11]:
l = 0.001
p = 0
windowsize = 10
for epoch in range(1, 200 + 1):
    if epoch%5 == 0:
        windowsize = windowsize+1
        p = p*1
        l = l*1
        if windowsize >= 20:
            windowsze = 20
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

lossMSE: 0.00649360241368413 lossSSIM: 0.2575039267539978
===> Epoch[1](0/676): Loss: 0.2575
lossMSE: 0.006390407215803862 lossSSIM: 0.25440138578414917
===> Epoch[1](50/676): Loss: 0.2544
lossMSE: 0.006614970974624157 lossSSIM: 0.26824480295181274
===> Epoch[1](100/676): Loss: 0.2682
lossMSE: 0.0075506893917918205 lossSSIM: 0.26137441396713257
===> Epoch[1](150/676): Loss: 0.2614
lossMSE: 0.006369191687554121 lossSSIM: 0.2585170865058899
===> Epoch[1](200/676): Loss: 0.2585
lossMSE: 0.00619878014549613 lossSSIM: 0.263511061668396
===> Epoch[1](250/676): Loss: 0.2635
lossMSE: 0.007016080431640148 lossSSIM: 0.2656596302986145
===> Epoch[1](300/676): Loss: 0.2657
lossMSE: 0.006515444256365299 lossSSIM: 0.2633575201034546
===> Epoch[1](350/676): Loss: 0.2634
lossMSE: 0.006589691620320082 lossSSIM: 0.2657267451286316
===> Epoch[1](400/676): Loss: 0.2657
lossMSE: 0.0072065615095198154 lossSSIM: 0.27124303579330444
===> Epoch[1](450/676): Loss: 0.2712
lossMSE: 0.006651228293776512 lossSSIM: 

===> Epoch 6 Complete: Avg. Loss: 0.2619
lossMSE: 0.006690953392535448 lossSSIM: 0.2620896100997925
===> Epoch[7](0/676): Loss: 0.2621
lossMSE: 0.007495889440178871 lossSSIM: 0.2672097682952881
===> Epoch[7](50/676): Loss: 0.2672
lossMSE: 0.007589578162878752 lossSSIM: 0.2699030637741089
===> Epoch[7](100/676): Loss: 0.2699
lossMSE: 0.005833015777170658 lossSSIM: 0.25563716888427734
===> Epoch[7](150/676): Loss: 0.2556
lossMSE: 0.006275692488998175 lossSSIM: 0.25534820556640625
===> Epoch[7](200/676): Loss: 0.2553
lossMSE: 0.005674084648489952 lossSSIM: 0.2575336694717407
===> Epoch[7](250/676): Loss: 0.2575
lossMSE: 0.0062295920215547085 lossSSIM: 0.25275105237960815
===> Epoch[7](300/676): Loss: 0.2528
lossMSE: 0.006321091670542955 lossSSIM: 0.25983482599258423
===> Epoch[7](350/676): Loss: 0.2598
lossMSE: 0.0061146412044763565 lossSSIM: 0.25911223888397217
===> Epoch[7](400/676): Loss: 0.2591
lossMSE: 0.006117822136729956 lossSSIM: 0.2617321014404297
===> Epoch[7](450/676): Loss: 0.

lossMSE: 0.006208880338817835 lossSSIM: 0.2597799301147461
===> Epoch[12](650/676): Loss: 0.2598
===> Epoch 12 Complete: Avg. Loss: 0.2613
lossMSE: 0.005852189380675554 lossSSIM: 0.2516065835952759
===> Epoch[13](0/676): Loss: 0.2516
lossMSE: 0.006552168633788824 lossSSIM: 0.26587730646133423
===> Epoch[13](50/676): Loss: 0.2659
lossMSE: 0.005699592642486095 lossSSIM: 0.2589109539985657
===> Epoch[13](100/676): Loss: 0.2589
lossMSE: 0.006968987174332142 lossSSIM: 0.2698909640312195
===> Epoch[13](150/676): Loss: 0.2699
lossMSE: 0.006052883807569742 lossSSIM: 0.25588148832321167
===> Epoch[13](200/676): Loss: 0.2559
lossMSE: 0.006225705146789551 lossSSIM: 0.26591062545776367
===> Epoch[13](250/676): Loss: 0.2659
lossMSE: 0.00680080009624362 lossSSIM: 0.2636934518814087
===> Epoch[13](300/676): Loss: 0.2637
lossMSE: 0.006636560428887606 lossSSIM: 0.25999557971954346
===> Epoch[13](350/676): Loss: 0.2600
lossMSE: 0.007141582202166319 lossSSIM: 0.2592223882675171
===> Epoch[13](400/676): L

KeyboardInterrupt: 

In [ ]:
l = 0.005
p = 0
windowsize = 12
for epoch in range(1, 200 + 1):
    if epoch%5 == 0:
        windowsize = windowsize+1
        p = p*1
        l = l*1
        if windowsize >= 16:
            windowsze = 16
    optimizer = optim.Adam(model.parameters(), lr=l)
        
    train(epoch, p, windowsize)

lossMSE: 0.007893196307122707 lossSSIM: 0.29469984769821167
===> Epoch[1](0/676): Loss: 0.2947
lossMSE: 0.007182117085903883 lossSSIM: 0.28323882818222046
===> Epoch[1](50/676): Loss: 0.2832
lossMSE: 0.006793836131691933 lossSSIM: 0.2764155864715576
===> Epoch[1](100/676): Loss: 0.2764
lossMSE: 0.007442289497703314 lossSSIM: 0.274247944355011
===> Epoch[1](150/676): Loss: 0.2742
lossMSE: 0.0061234417371451855 lossSSIM: 0.2635205388069153
===> Epoch[1](200/676): Loss: 0.2635
lossMSE: 0.007460900116711855 lossSSIM: 0.27335256338119507
===> Epoch[1](250/676): Loss: 0.2734
lossMSE: 0.0064965044148266315 lossSSIM: 0.26037561893463135
===> Epoch[1](300/676): Loss: 0.2604
lossMSE: 0.006868039723485708 lossSSIM: 0.2577994465827942
===> Epoch[1](350/676): Loss: 0.2578
lossMSE: 0.007156586740165949 lossSSIM: 0.2581159472465515
===> Epoch[1](400/676): Loss: 0.2581
lossMSE: 0.00600135512650013 lossSSIM: 0.25809985399246216
===> Epoch[1](450/676): Loss: 0.2581
lossMSE: 0.006207327824085951 lossSSIM

===> Epoch 6 Complete: Avg. Loss: 0.2642
lossMSE: 0.007071048021316528 lossSSIM: 0.26256710290908813
===> Epoch[7](0/676): Loss: 0.2626
lossMSE: 0.007511557079851627 lossSSIM: 0.26521849632263184
===> Epoch[7](50/676): Loss: 0.2652
lossMSE: 0.006430226843804121 lossSSIM: 0.2657700777053833
===> Epoch[7](100/676): Loss: 0.2658
lossMSE: 0.007603860925883055 lossSSIM: 0.2647833824157715
===> Epoch[7](150/676): Loss: 0.2648
lossMSE: 0.006077810190618038 lossSSIM: 0.2599230408668518
===> Epoch[7](200/676): Loss: 0.2599
lossMSE: 0.006454692222177982 lossSSIM: 0.26665204763412476
===> Epoch[7](250/676): Loss: 0.2667
lossMSE: 0.006814422085881233 lossSSIM: 0.25952303409576416
===> Epoch[7](300/676): Loss: 0.2595
lossMSE: 0.00816029030829668 lossSSIM: 0.2607625722885132
===> Epoch[7](350/676): Loss: 0.2608
lossMSE: 0.005881082266569138 lossSSIM: 0.25140881538391113
===> Epoch[7](400/676): Loss: 0.2514
lossMSE: 0.007026921957731247 lossSSIM: 0.261482298374176
===> Epoch[7](450/676): Loss: 0.2615

lossMSE: 0.00574009632691741 lossSSIM: 0.2526765465736389
===> Epoch[12](650/676): Loss: 0.2527
===> Epoch 12 Complete: Avg. Loss: 0.2622
lossMSE: 0.0078229159116745 lossSSIM: 0.2722821831703186
===> Epoch[13](0/676): Loss: 0.2723
lossMSE: 0.005926814395934343 lossSSIM: 0.25871825218200684
===> Epoch[13](50/676): Loss: 0.2587
lossMSE: 0.0073097944259643555 lossSSIM: 0.26126646995544434
===> Epoch[13](100/676): Loss: 0.2613
lossMSE: 0.006155519280582666 lossSSIM: 0.2589484453201294
===> Epoch[13](150/676): Loss: 0.2589
lossMSE: 0.006598445121198893 lossSSIM: 0.24903541803359985
===> Epoch[13](200/676): Loss: 0.2490
lossMSE: 0.006070698611438274 lossSSIM: 0.2588949203491211
===> Epoch[13](250/676): Loss: 0.2589
lossMSE: 0.006272806320339441 lossSSIM: 0.26031529903411865
===> Epoch[13](300/676): Loss: 0.2603
lossMSE: 0.006402990780770779 lossSSIM: 0.2557840943336487
===> Epoch[13](350/676): Loss: 0.2558
lossMSE: 0.006352097727358341 lossSSIM: 0.25905120372772217
===> Epoch[13](400/676): L

lossMSE: 0.0059328568167984486 lossSSIM: 0.26005852222442627
===> Epoch[18](550/676): Loss: 0.2601
lossMSE: 0.005903433542698622 lossSSIM: 0.2587625980377197
===> Epoch[18](600/676): Loss: 0.2588
lossMSE: 0.006827028002589941 lossSSIM: 0.25861090421676636
===> Epoch[18](650/676): Loss: 0.2586
===> Epoch 18 Complete: Avg. Loss: 0.2612
lossMSE: 0.00561586394906044 lossSSIM: 0.25061047077178955
===> Epoch[19](0/676): Loss: 0.2506
lossMSE: 0.006264632102102041 lossSSIM: 0.25821638107299805
===> Epoch[19](50/676): Loss: 0.2582
lossMSE: 0.006539321504533291 lossSSIM: 0.2606229782104492
===> Epoch[19](100/676): Loss: 0.2606
lossMSE: 0.00649147666990757 lossSSIM: 0.2559915781021118
===> Epoch[19](150/676): Loss: 0.2560
lossMSE: 0.006951265037059784 lossSSIM: 0.26227062940597534
===> Epoch[19](200/676): Loss: 0.2623
lossMSE: 0.005861871410161257 lossSSIM: 0.2605118155479431
===> Epoch[19](250/676): Loss: 0.2605
lossMSE: 0.0063163102604448795 lossSSIM: 0.26511311531066895
===> Epoch[19](300/676)

lossMSE: 0.0058670747093856335 lossSSIM: 0.2600933313369751
===> Epoch[24](450/676): Loss: 0.2601
lossMSE: 0.007442327216267586 lossSSIM: 0.2667045593261719
===> Epoch[24](500/676): Loss: 0.2667
lossMSE: 0.00633067823946476 lossSSIM: 0.2664073705673218
===> Epoch[24](550/676): Loss: 0.2664
lossMSE: 0.005778285674750805 lossSSIM: 0.2530285120010376
===> Epoch[24](600/676): Loss: 0.2530
lossMSE: 0.006617178674787283 lossSSIM: 0.262759804725647
===> Epoch[24](650/676): Loss: 0.2628
===> Epoch 24 Complete: Avg. Loss: 0.2602
lossMSE: 0.0066306437365710735 lossSSIM: 0.2643713355064392
===> Epoch[25](0/676): Loss: 0.2644
lossMSE: 0.00772565184161067 lossSSIM: 0.26634275913238525
===> Epoch[25](50/676): Loss: 0.2663
lossMSE: 0.007714930456131697 lossSSIM: 0.2720068693161011
===> Epoch[25](100/676): Loss: 0.2720
lossMSE: 0.006580830551683903 lossSSIM: 0.26124072074890137
===> Epoch[25](150/676): Loss: 0.2612
lossMSE: 0.006662558298557997 lossSSIM: 0.2564198970794678
===> Epoch[25](200/676): Los

...

lossMSE: 0.004961308557540178 lossSSIM: 0.23235654830932617
===> Epoch[106](450/676): Loss: 0.2324
lossMSE: 0.006183871533721685 lossSSIM: 0.2545727491378784
===> Epoch[106](500/676): Loss: 0.2546
lossMSE: 0.006013920996338129 lossSSIM: 0.24546706676483154
===> Epoch[106](550/676): Loss: 0.2455
lossMSE: 0.006178645417094231 lossSSIM: 0.24286407232284546
===> Epoch[106](600/676): Loss: 0.2429
lossMSE: 0.007079785689711571 lossSSIM: 0.2499421238899231
===> Epoch[106](650/676): Loss: 0.2499
===> Epoch 106 Complete: Avg. Loss: 0.2521
lossMSE: 0.006523297633975744 lossSSIM: 0.2460281252861023
===> Epoch[107](0/676): Loss: 0.2460
lossMSE: 0.007129027042537928 lossSSIM: 0.25684958696365356
===> Epoch[107](50/676): Loss: 0.2568
lossMSE: 0.006070664152503014 lossSSIM: 0.24533849954605103
===> Epoch[107](100/676): Loss: 0.2453
lossMSE: 0.00703168660402298 lossSSIM: 0.2580968141555786
===> Epoch[107](150/676): Loss: 0.2581
lossMSE: 0.006300698965787888 lossSSIM: 0.24846690893173218
===> Epoch[107

lossMSE: 0.006815820001065731 lossSSIM: 0.25153082609176636
===> Epoch[112](300/676): Loss: 0.2515
lossMSE: 0.006421705707907677 lossSSIM: 0.25922513008117676
===> Epoch[112](350/676): Loss: 0.2592
lossMSE: 0.006939607672393322 lossSSIM: 0.25945615768432617
===> Epoch[112](400/676): Loss: 0.2595
lossMSE: 0.006818902213126421 lossSSIM: 0.264070987701416
===> Epoch[112](450/676): Loss: 0.2641
lossMSE: 0.006383414845913649 lossSSIM: 0.2553509473800659
===> Epoch[112](500/676): Loss: 0.2554
lossMSE: 0.005648663267493248 lossSSIM: 0.24642246961593628
===> Epoch[112](550/676): Loss: 0.2464
lossMSE: 0.006779908202588558 lossSSIM: 0.25649547576904297
===> Epoch[112](600/676): Loss: 0.2565
lossMSE: 0.006458493880927563 lossSSIM: 0.2567020058631897
===> Epoch[112](650/676): Loss: 0.2567
===> Epoch 112 Complete: Avg. Loss: 0.2516
lossMSE: 0.006544663570821285 lossSSIM: 0.258614718914032
===> Epoch[113](0/676): Loss: 0.2586
lossMSE: 0.005886429455131292 lossSSIM: 0.2589607834815979
===> Epoch[113]

lossMSE: 0.006637989543378353 lossSSIM: 0.2481558918952942
===> Epoch[118](150/676): Loss: 0.2482
lossMSE: 0.006837560795247555 lossSSIM: 0.2574293613433838
===> Epoch[118](200/676): Loss: 0.2574
lossMSE: 0.006749131251126528 lossSSIM: 0.25170332193374634
===> Epoch[118](250/676): Loss: 0.2517
lossMSE: 0.006435430143028498 lossSSIM: 0.24964690208435059
===> Epoch[118](300/676): Loss: 0.2496
lossMSE: 0.006298360414803028 lossSSIM: 0.25845688581466675
===> Epoch[118](350/676): Loss: 0.2585
lossMSE: 0.006047524511814117 lossSSIM: 0.2507435083389282
===> Epoch[118](400/676): Loss: 0.2507
lossMSE: 0.005900024902075529 lossSSIM: 0.24630528688430786
===> Epoch[118](450/676): Loss: 0.2463
lossMSE: 0.006690572015941143 lossSSIM: 0.24504470825195312
===> Epoch[118](500/676): Loss: 0.2450
lossMSE: 0.006650304421782494 lossSSIM: 0.25385499000549316
===> Epoch[118](550/676): Loss: 0.2539
lossMSE: 0.0061059193685650826 lossSSIM: 0.24471479654312134
===> Epoch[118](600/676): Loss: 0.2447
lossMSE: 0.0

lossMSE: 0.006697957869619131 lossSSIM: 0.2525970935821533
===> Epoch[124](0/676): Loss: 0.2526
lossMSE: 0.006330894771963358 lossSSIM: 0.2585941553115845
===> Epoch[124](50/676): Loss: 0.2586
lossMSE: 0.006415670271962881 lossSSIM: 0.2562723755836487
===> Epoch[124](100/676): Loss: 0.2563
lossMSE: 0.005696008447557688 lossSSIM: 0.25309813022613525
===> Epoch[124](150/676): Loss: 0.2531
lossMSE: 0.005473759490996599 lossSSIM: 0.24616211652755737
===> Epoch[124](200/676): Loss: 0.2462
lossMSE: 0.006384073290973902 lossSSIM: 0.25593310594558716
===> Epoch[124](250/676): Loss: 0.2559
lossMSE: 0.006704156752675772 lossSSIM: 0.2552517056465149
===> Epoch[124](300/676): Loss: 0.2553
lossMSE: 0.0064052096568048 lossSSIM: 0.2532949447631836
===> Epoch[124](350/676): Loss: 0.2533
lossMSE: 0.006241957191377878 lossSSIM: 0.24768084287643433
===> Epoch[124](400/676): Loss: 0.2477
lossMSE: 0.006401346065104008 lossSSIM: 0.24578773975372314
===> Epoch[124](450/676): Loss: 0.2458
lossMSE: 0.006117655

lossMSE: 0.005781757645308971 lossSSIM: 0.24090951681137085
===> Epoch[129](600/676): Loss: 0.2409
lossMSE: 0.006294070277363062 lossSSIM: 0.25671738386154175
===> Epoch[129](650/676): Loss: 0.2567
===> Epoch 129 Complete: Avg. Loss: 0.2500
lossMSE: 0.005927528254687786 lossSSIM: 0.23757493495941162
===> Epoch[130](0/676): Loss: 0.2376
lossMSE: 0.0062964423559606075 lossSSIM: 0.24853235483169556
===> Epoch[130](50/676): Loss: 0.2485
lossMSE: 0.006270686164498329 lossSSIM: 0.2425815463066101
===> Epoch[130](100/676): Loss: 0.2426
lossMSE: 0.006004202179610729 lossSSIM: 0.24636274576187134
===> Epoch[130](150/676): Loss: 0.2464
lossMSE: 0.005983306560665369 lossSSIM: 0.24789053201675415
===> Epoch[130](200/676): Loss: 0.2479
lossMSE: 0.0062000565230846405 lossSSIM: 0.2512739896774292
===> Epoch[130](250/676): Loss: 0.2513
lossMSE: 0.006310439668595791 lossSSIM: 0.24851202964782715
===> Epoch[130](300/676): Loss: 0.2485
lossMSE: 0.006603749003261328 lossSSIM: 0.2478160262107849
===> Epoch